In [2]:
import csv
import base64
import time
import urllib
import traceback
from datetime import datetime, timedelta
from github import Github, StatsContributor
from awesomecure import awesome2py
from urllib.parse import urlparse
from os import getenv, path
from dotenv import load_dotenv
from termcolor import colored
import yaml

import requests
from bs4 import BeautifulSoup

In [3]:
def crawl_dependents(repo,page_num):
    url = 'https://github.com/{}/network/dependents'.format(repo)
    dependents_data = []
    list_end = False
    
    for i in range(page_num):
        #print("GET " + url)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")

        page_data = [
            "{}/{}".format(
                t.find('a', {"data-repository-hovercards-enabled":""}).text,
                t.find('a', {"data-hovercard-type":"repository"}).text
            )
            for t in soup.findAll("div", {"class": "Box-row"})
        ]
        
        for dependent in page_data:
            if dependent in dependents_data:
                list_end = True 
        
        if list_end:
            break
        else:    
            dependents_data = dependents_data + page_data
        
        try:
            paginationContainer = soup.find("div", {"class":"paginate-container"}).find_all('a')
        except:
            break
        
        try:
            if len(paginationContainer) > 1:
                paginationContainer = paginationContainer[1]
            else:
                paginationContainer = paginationContainer[0]
        except:
            break
        
        if paginationContainer:
            url = paginationContainer["href"]
        else:
            break
        
    return dependents_data

In [4]:
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = "{:02d}:{:02d}".format(mins, secs)
        print(timeformat, end="\r")
        time.sleep(1)
        t -= 1
    print("\n\n\n\n\n")

In [5]:
assert load_dotenv(), 'Environment variables could not be loaded'
g = Github(getenv("GITHUB"))

In [6]:
awesome_url = "https://github.com/protontypes/open-sustainable-technology"
awesome_path = urlparse(awesome_url).path.strip("/")
filename = "README.md"

awesome_repo = g.get_repo(awesome_path)
awesome_content_encoded = awesome_repo.get_contents(
    urllib.parse.quote(filename)
).content
awesome_content = base64.b64decode(awesome_content_encoded)

filehandle = open(".awesome.md", "w")
filehandle.write(awesome_content.decode("utf-8"))
filehandle.close()
repo_dict = awesome2py.AwesomeList(".awesome.md")

print(repo_dict)

Photovoltaics and Solar Energy
 - pvlib-python - A set of documented functions for simulating the performance of photovoltaic energy systems. [https://github.com/pvlib/pvlib-python] - pvfactors - Open source view-factor model for diffuse shading and bifacial PV modeling. [https://github.com/SunPower/pvfactors] - gsee - Global Solar Energy Estimator. [https://github.com/renewables-ninja/gsee] - PVMismatch - An explicit Python PV system IV & PV curve trace calculator which can also calculate mismatch. [https://github.com/SunPower/PVMismatch] - rdtools - An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems. [https://github.com/NREL/rdtools] - Machine-Learning-for-Solar-Energy-Prediction - Predict the power production of a solar panel farm from weather measurements using machine learning. [https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction] - elpv-dataset - A dataset of functional and defective solar c

 - Datasets APIs and open source projects related to Climate Change - A curated list of APIs, open data and ML/AI projects on climate change. [https://github.com/KKulma/climate-change-data] - Awesome Green Software - Research, tools, code, libraries and training to for building applications that emit less carbon into our atmosphere. [https://github.com/Green-Software-Foundation/awesome-green-software] - Awesome Sustainability Jobs - A curated list of companies in the sustainability sector that have jobs for devs. [https://github.com/pogopaule/awesome-sustainability-jobs] - Awesome Spectral Indices - A ready-to-use curated list of Spectral Indices for Remote Sensing applications. [https://github.com/davemlz/awesome-ee-spectral-indices] - Awesome Vegetation Index - List of reference, applications of common Vegetation Indices for Multi-spectral, hyper-spectral and UAV images. [https://github.com/px39n/Awesome-Vegetation-Index] - awesome-transit - Community list of transit APIs, apps, data

In [7]:
csv_projects = open("./csv/projects.csv", "w", newline="")
csv_github_organizations = open("./csv/github_organizations.csv", "r", newline="")

csv_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "platform",
    "topics",
    "rubric",
    "last_commit_date",
    "stargazers_count",
    "number_of_dependents",
    "stars_last_year",
    "project_active",
    "dominating_language",
    "organization",
    "organization_user_name",
    "languages",
    "homepage",
    "project_created",
    "project_age_in_days",
    "license",
    "total_commits_last_year",
    "total_number_of_commits",
    "last_issue_closed",
    "open_issues",
    "closed_pullrequests",
    "closed_issues",
    "issues_closed_last_year",
    "days_until_last_issue_closed",
    "open_pullrequests",
    "reviews_per_pr",
    "development_distribution_score",
    "last_released_date",
    "last_release_tag_name",
    "good_first_issue",
    "contributors",
    "accepts_donations",
    "donation_platforms",
    "code_of_conduct",
    "contribution_guide",
    "dependents_repos",
    "organization_name",
    "organization_github_url",
    "organization_website",
    "organization_location",
    "organization_country",
    "organization_form",
    "organization_avatar",
    "organization_public_repos",
    "organization_created",
    "organization_last_update",
]

csv_github_organizations_fieldnames = [
    "organization_name",
    "organization_user_name",
    "organization_github_url",
    "organization_website",
    "organization_location",
    "organization_country",
    "organization_form",
    "organization_avatar",
    "organization_public_repos",
    "organization_created",
    "organization_last_update",
    "organization_rubric"
]


writer_projects = csv.DictWriter(
    csv_projects, fieldnames=csv_fieldnames
)
writer_projects.writeheader()


github_organization_list=[]                            
reader_github_organizations = csv.DictReader(csv_github_organizations)
for entry in reader_github_organizations:
    github_organization_list.append(entry['organization_user_name'])
    
csv_github_organizations.close()
    
csv_github_organizations = open("./csv/github_organizations.csv", "a", newline="")
writer_github_organizations = csv.DictWriter(csv_github_organizations,csv_github_organizations_fieldnames)

In [8]:
retry = False
failures = []
min_requests = 100

for r in repo_dict.rubrics:
    for entry in r.entries:
        print("------------------------")
        print("Processing %s" % entry.name)
        print("URL: %s" % entry.url)
        if urlparse(entry.url).netloc == "github.com":
            print("%s is a GitHub project" % entry.name)
            while True:
                try:

                    remaining, limit = g.rate_limiting
                    resettime = g.rate_limiting_resettime
                    if remaining < min_requests:
                        print("------------------------")
                        print("Waiting for available GitHub requests:")
                        current_time = datetime.now().timestamp()
                        countdown((int(resettime) - int(current_time) + 2))

                    print(
                        "GitHub Requests | Limit: {}, Remaining: {}".format(
                            limit, remaining
                        )
                    )
                    # Gather project information from GitHub
                    # https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html
                    repo_path = urlparse(entry.url).path.strip("/")
                    platform = "github"
                    user, project_name = path.split(repo_path)
                    repo = g.get_repo(repo_path)
                    contents_root = repo.get_contents("")
                    releases = repo.get_releases()
                    commits = repo.get_commits()
                    stargazers = repo.get_stargazers_with_dates()

                    # Crawel dependents
                    try:
                        dependents_repos = crawl_dependents(repo_path,20)

                    except Exception as e:
                        print("Dependents not available:")
                        print(e)
                        dependents_repo = []


                    number_of_dependents = len(dependents_repos)
                    dependents_repos = ",".join(dependents_repos)

                    # Check if the project is still active
                    closed_issues = repo.get_issues(state="closed")
                    open_issues = repo.get_issues(state="open")

                    closed_pullrequests = repo.get_pulls(state="closed")
                    open_pullrequests = repo.get_pulls(state="open")



                    closed_prs = closed_pullrequests.totalCount
                    open_prs = open_pullrequests.totalCount

                    if closed_prs > 10:
                        pr_review_analyse = 10
                    else:
                        pr_review_analyse = closed_prs



                    total_reviews = 0     
                    for pull_request in closed_pullrequests[0:pr_review_analyse-1]:
                        pr_reviews = pull_request.get_reviews()
                        total_reviews = total_reviews + pr_reviews.totalCount
                    try:
                        reviews_per_pr = total_reviews/pr_review_analyse 
                    except:                   
                        reviews_per_pr = 0


                    if closed_issues.totalCount > 0:
                        last_issue_closed = closed_issues[0].updated_at
                        days_since_last_issue_closed = (datetime.now() - last_issue_closed).days


                    inactivity_time_delta = datetime.now() - timedelta(days=365)
                    issues_closed_time_delta = repo.get_issues(state="closed", since=inactivity_time_delta, sort="closed-desc")                

                    commits_time_delta = repo.get_commits(since=inactivity_time_delta)
                    last_commit_date = datetime.strptime(commits[0].last_modified, '%a, %d %b %Y %H:%M:%S GMT')


                    if (
                        issues_closed_time_delta.totalCount == 0
                        and commits_time_delta.totalCount == 0
                        or repo.archived
                    ):
                        print("%s is an inactive project" % entry.name)
                        project_active = "false"
                    else:
                        print("%s is an active project" % entry.name)
                        project_active = "true"



                    try:
                        license = repo.get_license()
                        if license.license.spdx_id == "NOASSERTION":
                            print("Custom license found")
                            license_name = "CUSTOM"
                        else:
                            license_name = license.license.spdx_id
                    except:
                        print("No license information found")
                        license_name == "UNDEFINED"

                    labels = ",".join([entry.name for entry in repo.get_labels()])
                    topics = ",".join(repo.get_topics())

                    languages_states = repo.get_languages()
                    programming_languages = ",".join(languages_states.keys())

                    try:
                        dominating_language = list(languages_states.keys())[0]

            #            try:
            #                readme = file.decoded_content
            #                if regexp.search(pip):
            #                    print("found pip")

            #                   if dominating_language = "Python":
            #                       pypi_project_stats = pypistats.overall("pyvista", total=True, format="pandas")
            #                       print(pypi_project_stats['downloads'])
            #                       print(pypi_project_stats.tail(1)['downloads'].values[0])
            #            except:
            #                print("nothing found")

                    except:
                        dominating_language = ""

                    try:
                        dotfolder_file = repo.get_contents(".github")
                        for file in dotfolder_file:
                            if file.path == ".github/FUNDING.yml":
                                print("Funding file found")
                                funding_file = base64.b64decode(file.content)
                                donation_platforms = ",".join(yaml.safe_load(funding_file))
                                accepts_donations = "true"
                                break
                            else:
                                donation_platforms = None
                                accepts_donations = "false"

                    except:
                        print("No funding information found")
                        donation_platforms = None
                        accepts_donations = "false"

                    try:
                        code_of_conduct = "false"
                        contribution_guide = "false"
                        for file_content in contents_root:
                            if file_content.path.lower().find("code_of_conduct") != -1:
                                print("Code of conduct found")
                                code_of_conduct = "true"
                            if file_content.path.lower().find("contributing") != -1:
                                print("Contribution guide found")
                                contribution_guide = "true"

                        for file_content in dotfolder_file:
                            if file_content.path.lower().find("code_of_conduct") != -1:
                                print("Code of conduct found")
                                code_of_conduct = "true"
                            if file_content.path.lower().find("contributing") != -1:
                                print("Contribution guide found")
                                contribution_guide = "true"

                    except Exception as e:
                        print(e)

                    contributors = repo.get_stats_contributors()
                    contributor_activity = {}
                    commits_total = 0
                    for individuum in contributors:
                        contributor_activity[individuum.author.login] = individuum.total
                        commits_total = commits_total + individuum.total

                    sorted_contributor = dict(
                        sorted(contributor_activity.items(), key=lambda item: item[1])
                    )
                    weighted_contribution = {
                        k: v / commits_total for k, v in sorted_contributor.items()
                    }

                    # Create a simple community health score that shows how much the project is focused on one developer
                    development_distribution_score = 1 - max(
                        weighted_contribution.values()
                    )

                    try:
                        last_released_date = releases[0].published_at.strftime(
                            "%Y/%m/%d, %H:%M:%S"
                        )
                        last_release_tag_name = releases[0].tag_name

                    except Exception as e:
                        print("No Release found")
                        last_released_date = ""
                        last_release_tag_name = ""
                        print(e)

                    total_number_of_commits = commits.totalCount

                    stars_last_year = 0
                    for star in stargazers:
                        starred_delta = datetime.utcnow() - star.starred_at
                        if  starred_delta < timedelta(days=365):
                            stars_last_year = stars_last_year + 1

                    # Gathering organization data
                    if repo.organization is None:
                        print("No Organization found. Project in user namespace.")
                        organization_user_name = None
                        organization_name = None
                        organization_avatar = None
                        organization_location = None
                        organization_github_url = None
                        organization_website = None
                        organization_created = None
                        organization_repos = None     
                        organization_last_update = None

                    elif repo.organization.login not in github_organization_list:
                        print("Organization not in list. Gathering data.")
                        organization_user_name = repo.organization.login
                        organization_name = repo.organization.name
                        organization_avatar = repo.organization.avatar_url
                        organization_location = repo.organization.location
                        organization_github_url = repo.organization.html_url
                        organization_website = repo.organization.blog
                        organization_created = repo.organization.created_at.strftime("%Y/%m/%d, %H:%M:%S")
                        organization_repos = g.search_repositories(query='user:'+organization_user_name, sort='updated')     
                        organization_last_update = organization_repos[0].updated_at.strftime("%Y/%m/%d, %H:%M:%S")


                        organization_data = {
                            "organization_name": organization_name,
                            "organization_user_name":organization_user_name,
                            "organization_github_url":organization_github_url,
                            "organization_website":organization_website,
                            "organization_avatar": organization_avatar,
                            "organization_location": organization_location,
                            "organization_country": "",
                            "organization_form": "",
                            "organization_public_repos": organization_repos.totalCount,
                            "organization_created": organization_created,
                            "organization_last_update": organization_last_update,
                            "organization_rubric": r.key
                        }

                        github_organization_list.append(organization_user_name)
                        print("Organization Data:")
                        print(organization_data)
                        writer_github_organizations.writerow(organization_data)

                    else:
                        organization_user_name = repo.organization.login
                        organization_name = repo.organization.name
                        organization_avatar = repo.organization.avatar_url
                        organization_location = repo.organization.location
                        organization_github_url = repo.organization.html_url
                        organization_website = repo.organization.blog                    

                    project_data = {
                        "project_name": entry.name,
                        "git_namespace": user,
                        "git_url": repo.clone_url,
                        "rubric": r.key,
                        "oneliner": entry.text[2:],
                        "topics": topics,
                        "organization": organization_name,
                        "organization_user_name": organization_user_name,
                        "project_created": repo.created_at.strftime("%Y/%m/%d, %H:%M:%S"),
                        "project_age_in_days": (datetime.now() - repo.created_at).days,
                        "last_commit_date": last_commit_date.strftime("%Y/%m/%d, %H:%M:%S"),
                        "project_active": project_active,
                        "last_issue_closed": last_issue_closed.strftime(
                            "%Y/%m/%d, %H:%M:%S"
                        ),
                        "last_released_date": last_released_date,
                        "last_release_tag_name": last_release_tag_name,
                        "total_number_of_commits": total_number_of_commits,
                        "total_commits_last_year": commits_time_delta.totalCount,
                        "development_distribution_score": development_distribution_score,
                        "stargazers_count": repo.stargazers_count,
                        "number_of_dependents": number_of_dependents,
                        "dependents_repos": dependents_repos,
                        "stars_last_year": stars_last_year,
                        "dominating_language": dominating_language,
                        "languages": programming_languages,
                        "homepage": repo.homepage,
                        "closed_issues": closed_issues.totalCount,
                        "issues_closed_last_year": issues_closed_time_delta.totalCount,
                        "days_until_last_issue_closed": days_since_last_issue_closed,
                        "open_issues": open_issues.totalCount,
                        "closed_pullrequests": closed_prs,
                        "open_pullrequests": open_prs,
                        "reviews_per_pr": reviews_per_pr,
                        "good_first_issue": repo.get_issues(
                            state="open", labels=["good first issue"]
                        ).totalCount,
                        "license": license_name,
                        "contributors": repo.get_contributors().totalCount,
                        "accepts_donations": accepts_donations,
                        "donation_platforms": donation_platforms,
                        "code_of_conduct": code_of_conduct,
                        "contribution_guide": contribution_guide,
                        "organization_avatar":organization_avatar,
                        "platform":platform,
                        "organization_github_url":organization_github_url,
                        "organization_website":organization_website,
                        "organization_avatar": organization_avatar,
                        "organization_location": organization_location,
                    }



                    print("Project Data:")
                    print(project_data)
                    writer_projects.writerow(project_data)
                    break

                except Exception as e:
                    print(colored("Failed to gather project information:"))
                    print(colored(e, "red"))
                    print(traceback.format_exc())
                    
                    if retry == False:
                        print(colored("Now try it one more time"))
                        retry = True
                        continue
                    else:
                        retry = False
                        print(colored("Last try. Now Quit"))
                        failures.append(entry.url)
                        break

        elif urlparse(entry.url).netloc == "gitlab.com":
            print("%s is a Gitlab project" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)
            platform = "gitlab"

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "git_url": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
                "platform":platform
            }
            print("Project Data:")
            print(project_data)
            writer_projects.writerow(project_data)

        else:
            print("%s is hosted on custom platform" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)
            platform = "custom"

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "homepage": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
                "platform":platform
            }

            print("Project Data:")
            print(project_data)
            writer_projects.writerow(project_data)

print("------------------------")
print("Processing finished. Saving CSV files")
print("Failure List:", failures)
csv_projects.close()
csv_github_organizations.close()

------------------------
Processing pvlib-python
URL: https://github.com/pvlib/pvlib-python
pvlib-python is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 265
pvlib-python is an active project
Code of conduct found
Contribution guide found
Project Data:


{'project_name': 'pvlib-python', 'git_namespace': 'pvlib', 'git_url': 'https://github.com/pvlib/pvlib-python.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A set of documented functions for simulating the performance of photovoltaic energy systems.', 'topics': 'solar-energy,python,renewable-energy,renewables,photovoltaic', 'organization': 'pvlib', 'organization_user_name': 'pvlib', 'project_created': '2015/02/17, 00:21:33', 'project_age_in_days': 2748, 'last_commit_date': '2022/08/19, 21:13:55', 'project_active': 'true', 'last_issue_closed': '2022/08/26, 23:09:21', 'last_released_date': '2022/08/19, 21:23:58', 'last_release_tag_name': 'v0.9.2', 'total_number_of_commits': 1422, 'total_commits_last_year': 73, 'development_distribution_score': 0.44664031620553357, 'stargazers_count': 723, 'number_of_dependents': 258, 'dependents_repos': 'openclimatefix/ocf_datapipes,pollination/sample-apps,flopaw/pvforecast-docker,moritz-reuter/ESEM-EE,PVSC-Python-Tutorials/PVPMC_2022,cwha

pvfactors is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 209
pvfactors is an active project
No funding information found
Contribution guide found
Project Data:
{'project_name': 'pvfactors', 'git_namespace': 'SunPower', 'git_url': 'https://github.com/SunPower/pvfactors.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Open source view-factor model for diffuse shading and bifacial PV modeling.', 'topics': 'solar-energy,renewable-energy,python,bifacial', 'organization': 'SunPower', 'organization_user_name': 'SunPower', 'project_created': '2018/05/14, 06:10:55', 'project_age_in_days': 1566, 'last_commit_date': '2022/02/22, 21:53:32', 'project_active': 'true', 'last_issue_closed': '2022/02/22, 21:53:38', 'last_released_date': '2022/02/22, 20:16:58', 'last_release_tag_name': 'v1.5.2', 'total_number_of_commits': 104, 'total_commits_last_year': 8, 'development_distribution_score': 0.12903225806451613, 'stargazers_count': 61, 'number_of_dependents': 7, 'dependents_re

{'project_name': 'Machine-Learning-for-Solar-Energy-Prediction', 'git_namespace': 'ColasGael', 'git_url': 'https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Predict the power production of a solar panel farm from weather measurements using machine learning.', 'topics': 'machine-learning,neural-network,data-processing,python,matlab,tensorflow', 'organization': None, 'organization_user_name': None, 'project_created': '2018/05/06, 19:43:04', 'project_age_in_days': 1573, 'last_commit_date': '2019/11/07, 18:37:29', 'project_active': 'false', 'last_issue_closed': '2019/11/15, 19:01:02', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 11, 'total_commits_last_year': 0, 'development_distribution_score': 0.0, 'stargazers_count': 141, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 32, 'dominating_language': 'Python', 'languages': 'Python,MATLAB,R,TeX,TSQL

{'project_name': 'pvcaptest', 'git_namespace': 'pvcaptest', 'git_url': 'https://github.com/pvcaptest/pvcaptest.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Collection of functions and Jupyter Notebooks to partially automate running a capacity test following ASTM E2848.', 'topics': '', 'organization': None, 'organization_user_name': 'pvcaptest', 'project_created': '2017/09/19, 01:33:23', 'project_age_in_days': 1803, 'last_commit_date': '2021/07/26, 13:43:21', 'project_active': 'true', 'last_issue_closed': '2021/07/25, 16:51:03', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 606, 'total_commits_last_year': 0, 'development_distribution_score': 0.0018832391713747842, 'stargazers_count': 13, 'number_of_dependents': 1, 'dependents_repos': 'pvcaptest/pvcaptest', 'stars_last_year': 2, 'dominating_language': 'Python', 'languages': 'Python', 'homepage': None, 'closed_issues': 45, 'issues_closed_last_year': 6, 'days_until_last_issue_closed': 3

...

{'project_name': 'Awesome Coastal', 'git_namespace': 'chrisleaman', 'git_url': 'https://github.com/chrisleaman/awesome-coastal.git', 'rubric': 'Curated Lists', 'oneliner': 'A curated list of awesome resources for coastal engineers and scientists.', 'topics': 'awesome,coastal,beaches,coastal-modelling,coastal-engineering', 'organization': None, 'organization_user_name': None, 'project_created': '2019/05/07, 23:47:21', 'project_age_in_days': 1207, 'last_commit_date': '2022/08/13, 05:53:52', 'project_active': 'true', 'last_issue_closed': '2022/08/13, 04:51:54', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 37, 'total_commits_last_year': 5, 'development_distribution_score': 0.1470588235294118, 'stargazers_count': 57, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 15, 'dominating_language': '', 'languages': '', 'homepage': '', 'closed_issues': 36, 'issues_closed_last_year': 28, 'days_until_last_issue_closed': 14, 'open_issues': 0, '

{'project_name': 'Awesome Clean Tech', 'git_namespace': 'nglgzz', 'git_url': 'https://github.com/nglgzz/awesome-clean-tech.git', 'rubric': 'Curated Lists', 'oneliner': 'Clean technology is any process, product, or service that reduces negative environmental impacts through significant energy efficiency improvements, the sustainable use of resources, or environmental protection activities.', 'topics': 'awesome,awesome-list,clean-tech,environment-friendly,climate-change,climate-change-mitigation', 'organization': None, 'organization_user_name': None, 'project_created': '2019/07/31, 19:54:41', 'project_age_in_days': 1123, 'last_commit_date': '2021/09/06, 07:12:06', 'project_active': 'true', 'last_issue_closed': '2021/09/06, 07:12:09', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 60, 'total_commits_last_year': 1, 'development_distribution_score': 0.5593220338983051, 'stargazers_count': 361, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_

{'project_name': 'Code against Climate Change', 'git_namespace': 'daviddao', 'git_url': 'https://github.com/daviddao/code-against-climate-change.git', 'rubric': 'Curated Lists', 'oneliner': 'A curated list of tech projects against climate change - hoping to inspire disruptive technological climate action.', 'topics': '', 'organization': None, 'organization_user_name': None, 'project_created': '2018/12/29, 23:37:54', 'project_age_in_days': 1336, 'last_commit_date': '2020/10/13, 13:09:34', 'project_active': 'false', 'last_issue_closed': '2020/10/13, 13:12:58', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 9, 'total_commits_last_year': 0, 'development_distribution_score': 0.5, 'stargazers_count': 172, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 33, 'dominating_language': '', 'languages': '', 'homepage': '', 'closed_issues': 3, 'issues_closed_last_year': 0, 'days_until_last_issue_closed': 683, 'open_issues': 0, 'closed_pullreque

{'project_name': 'awesome-earth-engine-apps', 'git_namespace': 'philippgaertner', 'git_url': 'https://github.com/philippgaertner/awesome-earth-engine-apps.git', 'rubric': 'Curated Lists', 'oneliner': 'An awesome list of all available Google Earth Engine Apps and user-specific App Galleries.', 'topics': 'google-earth-engine,earth-engine,apps', 'organization': None, 'organization_user_name': None, 'project_created': '2021/05/28, 14:41:43', 'project_age_in_days': 456, 'last_commit_date': '2021/11/05, 15:23:05', 'project_active': 'true', 'last_issue_closed': '2021/07/25, 22:46:02', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 92, 'total_commits_last_year': 1, 'development_distribution_score': 0.0, 'stargazers_count': 83, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 46, 'dominating_language': '', 'languages': '', 'homepage': '', 'closed_issues': 0, 'issues_closed_last_year': 0, 'days_until_last_issue_closed': 397, 'open_issues': 

{'project_name': 'Awesome Sentinel', 'git_namespace': 'Fernerkundung', 'git_url': 'https://github.com/Fernerkundung/awesome-sentinel.git', 'rubric': 'Curated Lists', 'oneliner': 'A curated list of awesome tools, tutorials and APIs related to data from the Copernicus Sentinel Satellites.', 'topics': 'science,research,open-data,satellite-imagery,space,awesome-list,hacktoberfest', 'organization': None, 'organization_user_name': None, 'project_created': '2016/10/21, 07:30:25', 'project_age_in_days': 2136, 'last_commit_date': '2022/08/14, 07:59:42', 'project_active': 'true', 'last_issue_closed': '2022/08/14, 07:34:10', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 137, 'total_commits_last_year': 8, 'development_distribution_score': 0.2142857142857143, 'stargazers_count': 431, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 60, 'dominating_language': '', 'languages': '', 'homepage': '', 'closed_issues': 34, 'issues_closed_last_year': 